In [ ]:
import pandas as pd
import numpy as np #

# Load Data

In [ ]:
df_holidays_events = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
df_oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
df_stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
df_trans = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')

df_train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv', index_col='id')
df_test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv', index_col='id')

# Merge Datasets and Add Some New Features from Date

In [ ]:
df_oil1 = df_oil.copy()
df_oil1 = df_oil1.ffill()

# copying of train data and merging other data
df_train1 = df_train.merge(df_oil1, on = 'date', how='left')
df_train1 = df_train1.merge(df_stores, on = 'store_nbr', how='left')
df_train1 = df_train1.merge(df_trans, on = ['date', 'store_nbr'], how='left')

date_data = pd.to_datetime(df_train1['date'])
df_train1['year'] = date_data.dt.year
df_train1['month'] = date_data.dt.month
df_train1['week'] = date_data.dt.isocalendar().week
df_train1['quarter'] = date_data.dt.quarter
df_train1['day_of_week'] = date_data.dt.day_name()

In [ ]:
# copying of test data and merging other data
df_test1 = df_test.merge(df_oil1, on = 'date', how='left')
df_test1 = df_test1.merge(df_stores, on = 'store_nbr', how='left')
df_test1 = df_test1.merge(df_trans, on = ['date', 'store_nbr'], how='left')

date_data = pd.to_datetime(df_test1['date'])
df_test1['year'] = date_data.dt.year
df_test1['month'] = date_data.dt.month
df_test1['week'] = date_data.dt.isocalendar().week
df_test1['quarter'] = date_data.dt.quarter
df_test1['day_of_week'] = date_data.dt.day_name()

# Fill NA

In [ ]:
nasum = df_train1.isnull().sum()
# amount of NANs before
nasum

In [ ]:
nasum = df_test1.isnull().sum()
# amount of NANs before
nasum

In [ ]:
df_train1.dcoilwtico = df_train1.dcoilwtico.ffill()
df_train1.dcoilwtico = df_train1.dcoilwtico.bfill()

df_train1['trans_mean_family'] = df_train1.groupby(['store_nbr','family'])['transactions'].transform('mean')
df_train1['transactions'] = df_train1['transactions'].fillna(df_train1['trans_mean_family'])

In [ ]:
df_test1.dcoilwtico = df_test1.dcoilwtico.ffill()
df_test1.dcoilwtico = df_test1.dcoilwtico.bfill()

df_test1['trans_mean_family'] = df_test1.groupby(['store_nbr','family'])['transactions'].transform('mean')
df_test1['transactions'] = df_test1['transactions'].fillna(df_test1['trans_mean_family'])

In [ ]:
nasum = df_train1.isnull().sum()
# amount of NANs after
nasum

In [ ]:
print(df_train1.columns)
print(df_train1.dtypes)

# Change types to Reduce Memory Usage

In [ ]:
for col in df_train1.columns:
    if col != 'sales':
        if df_train1[col].dtype == 'float64':
            df_train1[col] = df_train1[col].astype('float32')
            df_test1[col] = df_test1[col].astype('float32')
        if df_train1[col].dtype == 'int64':
            df_train1[col] = df_train1[col].astype('int8')
            df_test1[col] = df_test1[col].astype('int8')

print(df_train1.dtypes)

# Count holidays daily

In [ ]:
holidays_count_date = pd.DataFrame(df_holidays_events.date.value_counts(), dtype='int8')
holidays_count_date = holidays_count_date.reset_index()
holidays_count_date.rename(columns = {'index': 'date', 'date': 'count'}, inplace = True)
holidays_count_date

In [ ]:
df_train1.head()

In [ ]:
df_train1 = pd.merge(df_train1, holidays_count_date, how="left", on="date")
df_train1 = df_train1.fillna(0)
df_test1 = pd.merge(df_test1, holidays_count_date, how="left", on="date")
df_test1 = df_test1.fillna(0)
df_train1.head()

# Make Shifts

In [ ]:
train_gp = df_train1.sort_values('date').groupby(['family', 'store_nbr', 'date'], as_index=False)
train_gp1 = train_gp.agg({'sales':['mean']})
train_gp1.columns = ['family', 'store_nbr', 'date', 'sales']

#test_gp = df_test1.sort_values('date').groupby(['family', 'store_nbr', 'date'], as_index=False)
#test_gp1 = test_gp.agg({'sales':['mean']})
#test_gp1.columns = ['family', 'store_nbr', 'date', 'sales']

train_gp1.head()

In [ ]:
look_back = 3

In [ ]:
def shift_series(data1, look_back=3):
    for i in range(look_back):
        data1[f'shift t-{i+1} sales'] = data1['sales'].shift(i+1)
        
    data1 = data1.drop(['sales'], axis=1)

    return data1

train_gp1 = shift_series(train_gp1, look_back=look_back)
#test_gp1 = shift_series(test_gp1, look_back=look_back)
train_gp1 = train_gp1.fillna(0)
#test_gp1 = test_gp1.fillna(0)
train_gp1.head()

In [ ]:
#df_train1 = df_train1.merge(train_gp1, on =['family', 'store_nbr', 'date'], how='left')
#df_test1 = df_test1.merge(test_gp1, on =['family', 'store_nbr', 'date'], how='left')
df_train1

In [ ]:
#look_forward = 15

In [ ]:
#pred_gp = df_train1.sort_values('date').groupby(['family', 'store_nbr', 'date'], as_index=False)
#pred_gp1 = pred_gp.agg({'sales':['mean']})
#pred_gp1.columns = ['family', 'store_nbr', 'date', 'sales']
#pred_gp1.head()

In [ ]:
def predict_shift_series(data1, look_forward=16):
    for i in range(look_forward):
        data1[f'shift t+{i+1} sales'] = data1['sales'].shift(i-1)
        
    data1 = data1.drop(['sales'], axis=1)

    return data1

#pred_gp1 = predict_shift_series(pred_gp1, look_forward=look_forward)
#pred_gp1 = pred_gp1.fillna(0)
#pred_gp1.head()


In [ ]:
#y = df_train1.sort_values('date').groupby(['family', 'store_nbr', 'date'], as_index=False)
#y = y.agg({'sales':['mean']})
#y.columns = ['family', 'store_nbr', 'date', 'sales']
#y = y.merge(pred_gp1, on =['family', 'store_nbr', 'date'], how='left')
#y

In [ ]:
#y = y.sort_values('date').groupby(['date', 'store_nbr', 'family'], as_index=False)
#y = y.agg({'sales':['mean']})
#y.columns = ['date', 'store_nbr', 'family', 'sales']
#y

# Find Columns for Label Encoding

In [ ]:
df_train1['date'] = pd.to_datetime(df_train1['date'])
df_test1['date'] = pd.to_datetime(df_test1['date'])
df_train1.dtypes

In [ ]:
cat_cols = [cname for cname in df_train1.columns if df_train1[cname].dtype == 'object']
cat_cols

In [ ]:
# Prepare dataset to train network (supervised learning)
y = pd.DataFrame(df_train1.sales)
df_train1 = df_train1.drop(['sales'], axis=1)

In [ ]:
df_train1 = df_train1.drop('date', axis=1)
df_test1 = df_test1.drop('date', axis=1)

In [ ]:
from sklearn import preprocessing
enc = preprocessing.LabelEncoder()
for col in cat_cols:
    df_train1[col] = enc.fit_transform(df_train1[col].astype(str))
    df_test1[col] = enc.fit_transform(df_test1[col])

# Transform and Normalise Data

In [ ]:
import numpy as np
train_data = np.array(df_train1)
test_data = np.array(df_test1)

In [ ]:
from sklearn.preprocessing import QuantileTransformer

qt = QuantileTransformer(n_quantiles=300, output_distribution='uniform')
train_data = qt.fit_transform(train_data)
test_data = qt.transform(test_data)

qty = QuantileTransformer(n_quantiles=300, output_distribution='uniform')
y = qty.fit_transform(y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler()
dataset_train_sc = x_scaler.fit_transform(train_data)
dataset_test_sc = x_scaler.transform(test_data)

y_scaler = MinMaxScaler()
y_sc = y_scaler.fit_transform(y)

# Prepare Neural Network from Tensorflow/Keras 

In [ ]:
import tensorflow as tf

In [ ]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
#from tensorflow.keras.engine.input_layer import Input
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import BatchNormalization

In [ ]:
input_width = len(dataset_train_sc[0])
input_width

In [ ]:
def model_builder(lr):
    """Модель нейронной сети"""
    inputA = keras.Input(shape=(input_width))
    line = Reshape((input_width,1))(inputA)
    line = Conv1D(filters=1024, kernel_size=2, padding='same', activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.1)(line)
    
    #line = MaxPooling1D(pool_size=2)(line)
    
    #line = Conv1D(filters=1024, kernel_size=1, padding='same', activation='relu')(line)
    #line = BatchNormalization()(line)
    #line = Dropout(0.1)(line)
    
    #line = MaxPooling1D(pool_size=2)(line)
    
    line = Flatten()(line)
    
    line = Dense(1024, activation='relu')(line)
    line = Dense(512, activation='relu')(line)
    line = Dense(256, activation='relu')(line)
    #line = Dropout(0.3)(line)
    
    outputA = Dense(units=1)(line)
    model = Model(inputs=inputA, outputs=outputA)
    #model = keras.models.load_model('models/model2')
    model.compile(
        #loss = tf.keras.losses.MeanSquaredLogarithmicError(reduction="auto", name="mean_squared_logarithmic_error"),
        loss = 'msle',
        optimizer = Adam(lr=lr), metrics=['mae'],)
    return model

In [ ]:
lr=0.001
with strategy.scope():
    model = model_builder(lr)
#with strategy.scope():
#    model = load_model('../input/ss-tsf/best.h5')

In [ ]:
model.summary()

In [ ]:
checkpoint_filepath = 'best.h5'
save_model_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_best_only=True)

train/test split

In [ ]:
N_split = int(0.05 * len(dataset_train_sc))
dataset_sc_TRAIN = dataset_train_sc[:-N_split, :]
dataset_sc_VAL = dataset_train_sc[-N_split:, :]
y_TRAIN = y_sc[:-N_split]
y_VAL = y_sc[-N_split:]

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=0.0001, verbose=1, mode='min')

In [ ]:
batch_size = 8192
EPOCHS = 100
#EPOCHS = 1
model.fit(dataset_sc_TRAIN, y_TRAIN, validation_data=(dataset_sc_VAL, y_VAL), batch_size=batch_size, epochs=EPOCHS, callbacks=[save_model_callback, reduce_lr], shuffle=True)


In [ ]:
with strategy.scope():
    model = load_model('best.h5')
#with strategy.scope():
#    model = load_model('../input/ss-tsf/best_6.3584e-04-162.h5')

# Pseudo Labeling

Repeat real training

In [ ]:
preds = model.predict(dataset_test_sc)
pseudo_y_TRAIN = np.concatenate([y_TRAIN, preds], axis=0)
pseudo_x_TRAIN = np.concatenate([dataset_sc_TRAIN, dataset_test_sc], axis=0)
len(pseudo_y_TRAIN)

In [ ]:
batch_size = 8192
EPOCHS = 1
#EPOCHS = 1
model.fit(pseudo_x_TRAIN, pseudo_y_TRAIN, validation_data=(dataset_sc_VAL, y_VAL), batch_size=batch_size, epochs=EPOCHS, callbacks=[save_model_callback, reduce_lr], shuffle=True)


# 1

In [ ]:
big_x = np.concatenate([dataset_train_sc, dataset_test_sc], axis=0)
big_y = np.concatenate([y_sc, preds], axis=0)


In [ ]:
preds = model.predict(dataset_test_sc)
preds1 = y_scaler.inverse_transform(preds)
preds1 = qty.inverse_transform(preds1)
for i in range(len(preds1)): preds1[i] = 0 if preds1[i] < 0 else preds1[i]
sales = pd.DataFrame({'Id': df_test.index,'sales': preds1[:,0]})

In [ ]:
sales = sales.set_index('Id')

In [ ]:
sales

In [ ]:
df_train

In [ ]:
df_test_sales = pd.merge(df_test, sales, left_index=True, right_index=True)
df_test_sales = df_test_sales.fillna(0)
df_test_sales

In [ ]:
df_test_sales2 = df_test_sales.sort_values('date').groupby(['family', 'store_nbr', 'date'], as_index=False)
df_test_sales2 = df_test_sales2.agg({'sales':['mean']})
df_test_sales2.columns = ['family', 'store_nbr', 'date', 'sales']


In [ ]:
df_test_sales2

In [ ]:
look_back = 3
y_shifted_test = shift_series(df_test_sales2, look_back=look_back)
y_shifted_test = y_shifted_test.fillna(0)

In [ ]:
y_shifted_test

In [ ]:
y_shifted_sorted_id = pd.merge(df_test, y_shifted_test, how='left', on=['date', 'store_nbr', 'family'])
y_shifted_sorted_id = y_shifted_sorted_id.fillna(0)
y_shifted_sorted_id.index = np.arange(3000888, 3000888+len(y_shifted_sorted_id))
y_shifted_sorted_id
#y_shifted_sorted_id = y_shifted.sort_values('date').groupby(['date', 'store_nbr', 'family'], as_index=False)
#y_shifted_sorted_id = y_shifted_sorted_id.agg({'sales':['mean']})
#y_shifted_sorted_id.columns = ['family', 'store_nbr', 'date', 'sales']
#y_shifted_sorted_id

In [ ]:
cat_cols = ['family']
for col in cat_cols:
    #df_train1[col] = enc.fit_transform(df_train1[col].astype(str))
    y_shifted_sorted_id[col] = enc.fit_transform(y_shifted_sorted_id[col])
#y_shifted_test.index = np.arange(3000888, 3000888+len(y_shifted_test))

In [ ]:
y_shifted_sorted_id = y_shifted_sorted_id.drop('date', axis=1)
y_shifted_sorted_id

In [ ]:
df_test1.index = np.arange(3000888, 3000888+len(df_test1))
df_test1

In [ ]:
df_test_shifted_sales = pd.merge(df_test1, y_shifted_sorted_id, how='left', left_index=True, right_index=True)
df_test_shifted_sales = df_test_shifted_sales.drop(['store_nbr_y', 'family_y', 'onpromotion_y'], axis=1)
df_test_shifted_sales.rename(columns = {'store_nbr_x': 'store_nbr', 'family_x': 'family', 'onpromotion_x': 'onpromotion'}, inplace = True)
df_test_shifted_sales

In [ ]:
df_train1 = df_train1.merge(train_gp1, on =['family', 'store_nbr', 'date'], how='left')
df_test1 = df_test1.merge(test_gp1, on =['family', 'store_nbr', 'date'], how='left')
df_train1

In [ ]:
train_data = np.array(df_train1)
test_data = np.array(df_test_shifted_sales)

In [ ]:
#df_train1 = df_train1.merge(train_gp1, on =['family', 'store_nbr', 'date'], how='left')
#df_test1 = df_test1.merge(test_gp1, on =['family', 'store_nbr', 'date'], how='left')
df_train1

# Predict and Submit

In [ ]:
with strategy.scope():
    model = load_model('best.h5')
#with strategy.scope():
#    model = load_model('../input/ss-tsf/best_6.3584e-04-162.h5')

In [ ]:
preds = model.predict(dataset_test_sc)
preds1 = y_scaler.inverse_transform(preds)
preds1 = qty.inverse_transform(preds1)
for i in range(len(preds1)): preds1[i] = 0 if preds1[i] < 0 else preds1[i]

In [ ]:
output = pd.DataFrame({'Id': df_test.index,'sales': preds1[:,0]})
path = 'sample_submission.csv'
output.to_csv(path, index=False)
output 

# Stay Tuned. Work is going on. Please, UPVOTE